# Covariance matrix emulator

This notebook will run through similar exercises as the basic example, but will demonstrate that the emulator can successfully predict the off-diagonals.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import covariance_emulator as ce

## Basic covariance matrix with off-diagonals

Let's start with a basic covariance matrix with off-diagonals defined by
$$
C_0^{ij} = e^{-|i-j|}\,,
$$
this way it is always invertible.

In [3]:
ndim = 3 #matrices will be ndim X ndim
C0 = np.zeros((ndim,ndim))
for i in range(0,ndim):
    for j in range(i,ndim):
        C0[i,j] = C0[j,i] = np.exp(-np.fabs(i-j))

## Test 1 - linear scaling

The first test will have linear scaling with a paramter $C(A) = AC_0$.

In [4]:
parameters = np.arange(1,10)
Cs = [p*C0 for p in parameters]

In [5]:
Emu = ce.CovEmu(parameters, Cs)

In [6]:
test_parameter = 5.5
Ctrue = test_parameter*C0

In [7]:
Cpredicted = Emu.predict(test_parameter)
print(Cpredicted)

[[4.98984503 1.8356614  0.67530209]
 [1.8356614  5.66514712 2.08409116]
 [0.67530209 2.08409116 5.75653932]]


In [8]:
#Define a function to pull out just the off-diagonals and compute the fractional differences
def get_fracdiff_in_offdiag(C1, C2, ndim):
    D = C1 - C2
    out = np.zeros(int(ndim*(ndim-1)/2))
    k = 0
    for i in range(1,ndim):
        for j in range(0,i):
            out[k] = D[i,j]/C1[i,j]
            k+=1
    return out

In [9]:
diff = get_fracdiff_in_offdiag(Ctrue,Cpredicted,ndim)
print(diff)

[ 0.09275545  0.09275545 -0.03002675]


### Bad?

This is pretty bad, but remember that by default the emulator is building things with one principle component. Let's try more and see how it does.

In [10]:
Emu = ce.CovEmu(parameters, Cs, NPC_D=2, NPC_L=1)
Cpredicted = Emu.predict(test_parameter)
print("Fractional differences in off-diagonals:")
print(get_fracdiff_in_offdiag(Ctrue,Cpredicted,ndim))
print("C(A) predicted:")
print(Cpredicted)

Fractional differences in off-diagonals:
[0.00040085 0.00040085 0.00040085]
C(A) predicted:
[[5.49779531 2.02252587 0.74404569]
 [2.02252587 5.49779531 2.02252587]
 [0.74404569 2.02252587 5.49779531]]


### Interesting

This is interesting. Recalling that in the LDL Cholesky decompisition, D and L don't have *only* information about the diagonals and off-diagonals, respectively. In fact, they both D and L contain information about both sets of elements. What we see here, is that given the smooth behavior of the covariance matrix, the emulator is improved just by increasing the number of principle components for D, and not L. Increasing the number of principle components to describe L yields only a marginal increase in accuracy.

## Test 2 - peakiness

A covariance matrix property we can test is how "peaky" it is. This will be done by parameterizing the matrices like so:
$$
C^{ij}(A) = e^{-A|i-j|}C_0\,.
$$
In this way, matrices with larger parameters $A$ will be smoother than $C_0$.

In [11]:
def get_CA(A,C0):
    C_ = np.zeros((ndim,ndim))
    for i in range(0,ndim):
        for j in range(i,ndim):
            C_[i,j] = C_[j,i] = np.exp(-A*np.fabs(i-j))
    return C_*C0
Cs = [get_CA(p,C0) for p in parameters]
Ctrue = get_CA(test_parameter,C0)

In [12]:
Emu = ce.CovEmu(parameters, Cs, NPC_D=2, NPC_L=2)
Cpredicted = Emu.predict(test_parameter)
print("Fractional differences in off-diagonals:")
print(get_fracdiff_in_offdiag(Ctrue,Cpredicted,ndim))

Fractional differences in off-diagonals:
[-1.13456698e-01 -2.78558186e+02 -1.13909446e-01]


## Oh no! (oh... wait)

It looks like the emulator is doing poorly, but this is deceptive. The emulator is actually doing just fine, the problem is that it is trying to predict a very small number. The ratio of the diagonal to the most off-diagonal is 1e-6, and the emulator is predicting 1e-4. This is almost certaintly not going to matter for a real analysis. Encouragingly, when the ratio is 1e-3, the prediction is at the 10% level. So it would appear that things are working fine.

In [13]:
print(get_fracdiff_in_offdiag(Ctrue,Cpredicted,ndim))

[-1.13456698e-01 -2.78558186e+02 -1.13909446e-01]


In [14]:
print(Ctrue)

[[1.00000000e+00 1.50343919e-03 2.26032941e-06]
 [1.50343919e-03 1.00000000e+00 1.50343919e-03]
 [2.26032941e-06 1.50343919e-03 1.00000000e+00]]


In [15]:
print(Cpredicted)

[[1.00000000e+00 1.67401444e-03 6.31893588e-04]
 [1.67401444e-03 9.99777523e-01 1.67469512e-03]
 [6.31893588e-04 1.67469512e-03 9.99777922e-01]]


## Test 3 - scaled off diagonals

In the previous examples, the scaling effected both the diagonals and off diagonals. Let's try something new with
$$
C^{ii}(A) = \bf{I}
$$
$$
C^{i\neq j}(A) = A\times10^{-2}\,.
$$
Of course, we have to restrict ourselves to $A<100$ in order to have valid covariances.

In [16]:
def get_CA(A,C0):
    C = np.ones((ndim,ndim))
    for i in range(0,ndim):
        for j in range(i,ndim):
            if i == j:
                C[i,j] = 1
            else:
                C[i,j] = C[j,i] = A*1e-2
    return C
Cs = [get_CA(p,C0) for p in parameters]
Ctrue = get_CA(test_parameter,C0)

In [17]:
Emu = ce.CovEmu(parameters, Cs, NPC_D=2, NPC_L=2)
Cpredicted = Emu.predict(test_parameter)
print("Fractional differences in off-diagonals:")
print(get_fracdiff_in_offdiag(Ctrue,Cpredicted,ndim))

Fractional differences in off-diagonals:
[-1.26820996e-05 -1.26820996e-05  2.44769506e-05]


### Hurray!

The emulator worked great. With two principle components to characterize both D and L, the emulator successfully predicted the off diagonal elements without issue at high precision.